In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd, numpy as np
from keras.utils import plot_model
from keras.models import Sequential
from matplotlib import pyplot as plt
from keras.layers import Flatten, InputLayer, Dense, Conv2D, MaxPooling2D, Dropout

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

train_images = train_images/255
test_images = test_images/255

In [ ]:
plt.imshow(train_images[1], cmap = 'gray')
plt.axis('off')
plt.show()

In [ ]:
x, y, z = np.expand_dims(train_images[1], -1).nonzero()
ax = plt.axes(projection='3d')
ax.scatter(x, y, z, c='k', alpha=1)
plt.show()

In [ ]:
from plotly import graph_objects as go

data = pd.DataFrame(train_images[1])
colors = [[0, 'white'], [0.5, 'gray'], [1, 'black']]
fig = go.Figure(data=[go.Surface(z=data, colorscale = colors)])
fig.show()

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(28, 28, 1)))
model.add(Conv2D(32, kernel_size=(4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten()
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(
    optimizer='adam',
    metrics=['accuracy'],
    loss='sparse_categorical_crossentropy'
)

In [ ]:
metrics = model.fit(
    train_images,
    train_labels,
    epochs = 10,
    validation_split = 0.2
)

In [ ]:
labels_prob = model.predict(test_images)
labels_pred = labels_prob.argmax(axis=1)

indices = np.where(test_labels != labels_pred)
err_pred = np.array(indices).reshape(-1)

In [ ]:
df = pd.DataFrame(metrics.history)
print(df)

In [ ]:
plt.plot(metrics.history['loss'])
plt.plot(metrics.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
plt.plot(metrics.history['accuracy'])
plt.plot(metrics.history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])
plt.show()

In [ ]:
model.save('mnist_model.h5')

In [ ]:
loaded_model = keras.models.load_model('mnist_model.h5')

In [ ]:
import os
from PIL import Image, ImageOps

for filename in os.listdir(os.getcwd()):
  if filename.endswith('.png'):
    image = Image.open(filename)
    image = image.convert('L')
    inverted_image = ImageOps.invert(image)
    imageAsArray = np.array(inverted_image)

    imageAsArray = imageAsArray / 255
    
    prediction = loaded_model.predict(np.expand_dims(imageAsArray, 0))
    plt.title('Label: %d' % np.argmax(prediction))
    plt.imshow(imageAsArray, cmap='gray')
    plt.axis('off')
    plt.show()